In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../../')
from src.data.preprocessing import load_audio, normalize_audio
from src.features.extractor import extract_mel_spectrogram, extract_mfcc
from src.utils.config import load_config

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 1. Load Configuration and Metadata

In [ ]:
# Load configuration
config = load_config('../../configs/config.yaml')

# Load metadata
metadata_path = os.path.join('../../', config['data']['data_dir'], 'meta', 'esc50.csv')
df = pd.read_csv(metadata_path)

print(f"Total samples in ESC-50: {len(df)}")
print(f"\nDataset columns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Filter for animal sounds only
animal_categories = config['data']['animal_categories']
df_animals = df[df['category'].isin(animal_categories)].copy()

print(f"Animal sound samples: {len(df_animals)}")
print(f"\nAnimal categories: {sorted(animal_categories)}")
print(f"\nSamples per category:")
print(df_animals['category'].value_counts().sort_index())

## 2. Dataset Distribution Analysis

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
category_counts = df_animals['category'].value_counts().sort_index()
axes[0].bar(range(len(category_counts)), category_counts.values, color='steelblue', alpha=0.8)
axes[0].set_xlabel('Animal Category', fontsize=12)
axes[0].set_ylabel('Number of Samples', fontsize=12)
axes[0].set_title('Animal Sound Distribution', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(len(category_counts)))
axes[0].set_xticklabels(category_counts.index, rotation=45, ha='right')
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
colors = plt.cm.Set3(range(len(category_counts)))
axes[1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
            colors=colors, startangle=90)
axes[1].set_title('Category Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Listen to Audio Samples

Let's listen to one sample from each animal category.

In [ ]:
# Select one sample from each category
data_dir = '../../' + config['data']['data_dir']

for category in sorted(animal_categories)[:3]:  # First 3 for demo
    sample = df_animals[df_animals['category'] == category].iloc[0]
    audio_path = os.path.join(data_dir, 'audio', sample['filename'])
    
    print(f"\n{'='*60}")
    print(f"Category: {category.upper()}")
    print(f"Filename: {sample['filename']}")
    print(f"{'='*60}")
    
    # Load and play audio
    audio, sr = librosa.load(audio_path, sr=config['data']['sample_rate'])
    display(Audio(audio, rate=sr))

## 4. Waveform Visualization

Visualize the time-domain representation of audio signals.

In [ ]:
# Plot waveforms for different animals
fig, axes = plt.subplots(5, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, category in enumerate(sorted(animal_categories)):
    sample = df_animals[df_animals['category'] == category].iloc[0]
    audio_path = os.path.join(data_dir, 'audio', sample['filename'])
    
    audio, sr = librosa.load(audio_path, sr=config['data']['sample_rate'], duration=5)
    
    librosa.display.waveshow(audio, sr=sr, ax=axes[idx], color='steelblue')
    axes[idx].set_title(f'{category.capitalize()}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Time (s)')
    axes[idx].set_ylabel('Amplitude')

plt.tight_layout()
plt.show()

## 5. Spectrogram Analysis

Spectrograms show how the frequency content of audio changes over time.

In [ ]:
# Compare spectrograms of 4 different animals
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

selected_animals = ['dog', 'cat', 'frog', 'rooster']

for idx, category in enumerate(selected_animals):
    sample = df_animals[df_animals['category'] == category].iloc[0]
    audio_path = os.path.join(data_dir, 'audio', sample['filename'])
    
    audio, sr = librosa.load(audio_path, sr=config['data']['sample_rate'], duration=5)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    img = librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', 
                                    ax=axes[idx], cmap='viridis')
    axes[idx].set_title(f'Spectrogram - {category.capitalize()}', fontsize=13, fontweight='bold')
    fig.colorbar(img, ax=axes[idx], format='%+2.0f dB')

plt.tight_layout()
plt.show()

## 6. Mel Spectrogram Features

Mel spectrograms use a perceptually-motivated scale that better represents how humans hear.

In [ ]:
# Compare mel spectrograms
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, category in enumerate(selected_animals):
    sample = df_animals[df_animals['category'] == category].iloc[0]
    audio_path = os.path.join(data_dir, 'audio', sample['filename'])
    
    audio = load_audio(audio_path, sample_rate=config['data']['sample_rate'], duration=5)
    mel_spec = extract_mel_spectrogram(
        audio,
        sample_rate=config['data']['sample_rate'],
        n_mels=config['features']['n_mels'],
        n_fft=config['features']['n_fft'],
        hop_length=config['features']['hop_length']
    )
    
    img = librosa.display.specshow(mel_spec, sr=config['data']['sample_rate'], 
                                    x_axis='time', y_axis='mel', ax=axes[idx], cmap='magma')
    axes[idx].set_title(f'Mel Spectrogram - {category.capitalize()}', fontsize=13, fontweight='bold')
    fig.colorbar(img, ax=axes[idx], format='%+2.0f dB')

plt.tight_layout()
plt.show()

## 7. MFCC Features

MFCCs (Mel-Frequency Cepstral Coefficients) are compact representations commonly used in speech and audio recognition.

In [ ]:
# Compare MFCCs
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, category in enumerate(selected_animals):
    sample = df_animals[df_animals['category'] == category].iloc[0]
    audio_path = os.path.join(data_dir, 'audio', sample['filename'])
    
    audio = load_audio(audio_path, sample_rate=config['data']['sample_rate'], duration=5)
    mfcc = extract_mfcc(
        audio,
        sample_rate=config['data']['sample_rate'],
        n_mfcc=config['features']['n_mfcc'],
        n_fft=config['features']['n_fft'],
        hop_length=config['features']['hop_length']
    )
    
    img = librosa.display.specshow(mfcc, sr=config['data']['sample_rate'], 
                                    x_axis='time', ax=axes[idx], cmap='coolwarm')
    axes[idx].set_title(f'MFCC - {category.capitalize()}', fontsize=13, fontweight='bold')
    axes[idx].set_ylabel('MFCC Coefficients')
    fig.colorbar(img, ax=axes[idx])

plt.tight_layout()
plt.show()

## 8. Feature Statistics Comparison

Compare statistical properties of features across different animal categories.

In [ ]:
# Extract features for multiple samples and compare statistics
feature_stats = []

for category in sorted(animal_categories):
    # Get first 5 samples from each category
    samples = df_animals[df_animals['category'] == category].head(5)
    
    for _, sample in samples.iterrows():
        audio_path = os.path.join(data_dir, 'audio', sample['filename'])
        
        try:
            audio = load_audio(audio_path, sample_rate=config['data']['sample_rate'], duration=5)
            
            # Extract MFCC
            mfcc = extract_mfcc(audio, sample_rate=config['data']['sample_rate'])
            
            feature_stats.append({
                'category': category,
                'mfcc_mean': mfcc.mean(),
                'mfcc_std': mfcc.std(),
                'mfcc_max': mfcc.max(),
                'mfcc_min': mfcc.min(),
                'zero_crossing_rate': librosa.feature.zero_crossing_rate(audio).mean(),
                'spectral_centroid': librosa.feature.spectral_centroid(y=audio, sr=config['data']['sample_rate']).mean(),
                'spectral_rolloff': librosa.feature.spectral_rolloff(y=audio, sr=config['data']['sample_rate']).mean()
            })
        except:
            continue

feature_df = pd.DataFrame(feature_stats)
print("\nFeature Statistics Summary:")
feature_df.groupby('category').mean().round(3)

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# MFCC mean
feature_df.boxplot(column='mfcc_mean', by='category', ax=axes[0, 0])
axes[0, 0].set_title('MFCC Mean Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Animal Category')
axes[0, 0].set_ylabel('MFCC Mean')
plt.setp(axes[0, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Zero Crossing Rate
feature_df.boxplot(column='zero_crossing_rate', by='category', ax=axes[0, 1])
axes[0, 1].set_title('Zero Crossing Rate', fontweight='bold')
axes[0, 1].set_xlabel('Animal Category')
axes[0, 1].set_ylabel('ZCR')
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Spectral Centroid
feature_df.boxplot(column='spectral_centroid', by='category', ax=axes[1, 0])
axes[1, 0].set_title('Spectral Centroid', fontweight='bold')
axes[1, 0].set_xlabel('Animal Category')
axes[1, 0].set_ylabel('Frequency (Hz)')
plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Spectral Rolloff
feature_df.boxplot(column='spectral_rolloff', by='category', ax=axes[1, 1])
axes[1, 1].set_title('Spectral Rolloff', fontweight='bold')
axes[1, 1].set_xlabel('Animal Category')
axes[1, 1].set_ylabel('Frequency (Hz)')
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.suptitle('')
plt.tight_layout()
plt.show()

## 9. Key Insights

### Observations:
1. **Class Balance**: All animal categories have equal representation (40 samples each)
2. **Waveform Patterns**: Different animals show distinct temporal patterns
3. **Frequency Characteristics**: 
   - Low-frequency animals: Cow, Pig, Frog
   - High-frequency animals: Insects, Rooster
4. **Feature Separability**: Mel spectrograms and MFCCs show clear differences between categories

### Next Steps:
1. Train CNN model on mel spectrogram features
2. Experiment with data augmentation
3. Try different model architectures (LSTM, Transformer)
4. Analyze misclassification patterns

## 10. Export Sample Visualizations

In [ ]:
# Save a comprehensive visualization for presentation
output_dir = '../../outputs/notebook_visuals'
os.makedirs(output_dir, exist_ok=True)

# Create a summary figure
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Add various plots to the summary
# ... (code to create summary visualization)

print(f"✓ Visualizations saved to {output_dir}")